### Python setup

See `README.md` for python environment setup instructions on OSX using `pyenv` in VSCode.  If you've got another way of getting a modern, viable python environment, feel free to use it.

Here are the versions that I'm using when running this notebook:

In [1]:
import platform
import psutil

print(platform.python_version())

memory_info = psutil.virtual_memory()
print(f"Total memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Memory percent: {memory_info.percent}%")

3.12.2
Total memory: 64.00 GB
Available memory: 33.31 GB
Used memory: 29.83 GB
Memory percent: 48.0%


### Generate a self-signed cert that we can use for the https redirect during the OAuth login

You'll need to tell your browser to "trust" this cert after the redirect

In [2]:
import os
import subprocess
import shutil
from IPython import get_ipython

def generate_key():
    if shutil.which("openssl") is None:
        print("openssl is not found. Please install openssl and make sure it's on your system path. See: https://wiki.openssl.org/index.php/Binaries")
        return

    if os.path.isfile("key.pem"):
        print("key.pem already exists, not regenerating it. Delete it if you'd like to regenerate it.")
    else:
        command = 'openssl req -x509 -newkey rsa:4096 -keyout key.pem -out cert.pem -days 365 -nodes -subj "/C=US/ST=California/L=San Francisco/O=My Company/OU=My Division/CN=localhost"'
        process = subprocess.Popen(command, shell=True)
        process.wait()

generate_key()

key.pem already exists, not regenerating it. Delete it if you'd like to regenerate it.


### API Key and Client ID

You'll need an API Key and Client ID to interact with Bungie's API.  You can generate one on Bungie's website: https://www.bungie.net/en/Application

You can generate an application with the "Public" OAuth type with a redirect url of: `https://localhost:7777/oauth-redirect`  - this will be used after the oauth login to get the oauth token that will be used in API requests

The only scope necessary is read access: `Read your Destiny 2 information (Vault, Inventory, and Vendors), as well as Destiny 1 Vault and Inventory data` 

This is what it should look like:

![Image](images/oauth-app-settings.png)


Once you create your application, you should get an `OAuth client_id` and an `API Key`, values for these should be put into `config.json` along with your Bungie `username` (Steam usernames will have a 4-digit hash like `#1234` on the end).  It should look like this:

```
{ "client_id": "your_client_id", "api_key": "your_api_key", "username": "your_username#1234"}
```

This same username is used on things like https://dungeon.report to find your user.

This next cell will read that file in and make the `client_id`, `api_key`, and `username` values available to the rest of the calls.


In [3]:
from src import config

client_id, api_key, username = config.load_config()

print(f"client_id length: {len(client_id)}")
print(f"api_key length: {len(api_key)}")
print(f"username: {username}")

client_id length: 5
api_key length: 32
username: Dera#9169


### Login with OAuth to get a token that'll last for 1 hour

In [4]:
from src.bungie_oauth import BungieAuth
import datetime

# perform oauth login to get the access token used in later requests.  It is good for 1 hour
print("We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser.")
access_token = BungieAuth(client_id).refresh_oauth_token()

# token is good for 1 hour, print out the time that it expires
expiration_time = datetime.datetime.now() + datetime.timedelta(hours=1)
print(f"Access token successfully acquired at: {datetime.datetime.now().isoformat()} and expires at: {expiration_time.isoformat()}")

We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser.
Please go to the following URL and authorize the app: https://www.bungie.net/en/oauth/authorize?client_id=47118&response_type=code&state=rmQr13l7FAvGgasgdhi0zKpxJUI7QZFvo7Z7ak9TnOlxlYb1DtVhf_6NYeqVRqYRrd8mNRQdiILH_cTdS3y9ZYzzg5DgoX7v2zXPuDpTfRuyw75JqG-pP5P0inJDs4sMXKv_mp3tO_X0zgHSOjGxKegiGHa_bZqq5wLMD9N8jqI=&redirect_uri=https://localhost:7777/
Stopping HTTPS server
Access token successfully acquired at: 2024-06-13T08:07:38.550028 and expires at: 2024-06-13T09:07:38.549980


### Now we're ready to talk to Bungie's API

In [5]:
from src.bungie_api import BungieApi

api = BungieApi(api_key, access_token)

membership_id, profile_type = api.get_primary_membership_id_and_type(username)
print(f"Membership ID: {membership_id}, Profile Type: {profile_type}")

api.get_character_ids_and_classes(membership_id, profile_type)

Checking membership ID 4611686018465007625 with membership type 2
Crosave override found for 4611686018465007625
Membership ID: 4611686018465007625, Profile Type: 2


{'2305843009262309873': 'Warlock',
 '2305843009262309875': 'Titan',
 '2305843009716994332': 'Hunter'}

In [20]:
# retrieve the manifest and item/stat definitions that will be joined with profile data to determine what armor you have in your vault
item_definitions, stat_definitions = api.get_static_definitions()

In [18]:
manifest = api.get_manifest()
from pprint import pprint

world_component_content_paths = manifest['Response']['jsonWorldComponentContentPaths']['en']
print("jsonWorldComponentContentPaths:")
for key, value in world_component_content_paths.items():
    print(f"  {key}: {value}")

world_content_paths = manifest['Response']['jsonWorldContentPaths']['en']
print(f"jsonWorldContentPaths: {world_content_paths}")

mobile_asset_content_path = manifest['Response']['mobileAssetContentPath']
print(f"mobileAssetContentPath: {mobile_asset_content_path}")

mobile_gear_asset_data_bases = manifest['Response']['mobileGearAssetDataBases']
print(f"mobileGearAssetDataBases:")
for db in mobile_gear_asset_data_bases:
    print(f"  {db['version']}: {db['path']}")

mobile_world_content_paths = manifest['Response']['mobileWorldContentPaths']['en']
print(f"mobileWorldContentPaths: {mobile_world_content_paths}")


jsonWorldComponentContentPaths:
  DestinyNodeStepSummaryDefinition: /common/destiny2_content/json/en/DestinyNodeStepSummaryDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyArtDyeChannelDefinition: /common/destiny2_content/json/en/DestinyArtDyeChannelDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyArtDyeReferenceDefinition: /common/destiny2_content/json/en/DestinyArtDyeReferenceDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyPlaceDefinition: /common/destiny2_content/json/en/DestinyPlaceDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyActivityDefinition: /common/destiny2_content/json/en/DestinyActivityDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyActivityTypeDefinition: /common/destiny2_content/json/en/DestinyActivityTypeDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyClassDefinition: /common/destiny2_content/json/en/DestinyClassDefinition-f946ef41-1dd2-49fb-bd7e-a3c45b9d20e5.json
  DestinyGenderDefinitio

In [7]:
# download the character profile for this membership_id
import os
import json

# access_token, profile_type, and membership_id should be retrieved above 
# using the login_and_get_token and get_primary_membership_id_and_type functions

# https://bungie-net.github.io/multi/schema_Destiny-DestinyComponentType.html#schema_Destiny-DestinyComponentType
# 100 = profile.data.userInfo
# 102 = profileInventory.data.items
# 201 = characterInventories.data[character_id].items
# 205 = characterEquipment.data[character_id].items
# 300 = itemComponents.instances
# 305 = profilePlugSets.data.plugs, characterPlugSets.data[character_id].plugs, itemComponents.sockets

# this gives us all of the information we need for vault armor for this user
profile = api.get_profile(access_token, profile_type, membership_id, [100,102,201,205,300,305,309])

os.makedirs('data', exist_ok=True)

# dump the profile out as json into the data directory
with open('data/profile.json', 'w') as file:
    json.dump(profile, file, indent=4)

print(f"Character profile loaded at:", profile["responseMintedTimestamp"])

Character profile loaded at: 2024-06-04T23:03:08.12Z


In [14]:
print("Print tree of up to second level keys in profile:")
for key in profile.keys():
    print(key)
    if isinstance(profile[key], dict):
        for subkey in profile[key].keys():
            print(f"  {subkey}")


Print tree of up to second level keys in profile:
responseMintedTimestamp
secondaryComponentsMintedTimestamp
profileInventory
  data
  privacy
profile
  data
  privacy
profilePlugSets
  data
  privacy
characterInventories
  data
  privacy
characterEquipment
  data
  privacy
characterPlugSets
  data
  privacy
itemComponents
  instances
  sockets
  plugObjectives


In [8]:
# extract all armor pieces out of the profile.  It retrieves from the vault, character inventory, and character equipment
import src.armor as armor
import pandas as pd

profile_armor = armor.ProfileArmor(profile, item_definitions, stat_definitions)

armor_dict = profile_armor.get_armor_dict()

armor_df = pd.DataFrame([{
    **vars(armor), 
    'total_stats': armor.total_stats,
    'is_exotic': armor.is_exotic,
    'class_slot': armor.class_slot
} for armor in armor_dict.values()])

armor_df

,item_name,item_hash,instance_id,rarity,slot,power,mobility,resilience,recovery,discipline,intellect,strength,is_artifice,is_masterworked,d2_class,total_stats,is_exotic,class_slot
0,Iron Forerunner Hood,2217519207,6917529839573126708,Legendary,Helmet,1900,6,19,6,7,12,14,False,False,Warlock,64,False,Warlock Helmet
1,Grasp of Eir,3398467185,6917529814420019592,Legendary,Gauntlets,1900,9,21,2,16,6,12,False,True,Warlock,66,False,Warlock Gauntlets
2,Phoenix Protocol,4057299719,6917529106848181107,Exotic,Chest Armor,1900,15,6,8,15,14,2,False,True,Warlock,60,True,Warlock Chest Armor
3,Boots of Detestation,3702434452,6917529883188092920,Legendary,Leg Armor,1900,2,22,7,13,15,2,False,False,Warlock,61,False,Warlock Leg Armor
4,Reverie Dawn Bond,1394177923,6917529814054890031,Legendary,Class Item,1900,0,0,0,0,0,0,False,True,Warlock,0,False,Warlock Class Item
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Parade Greaves,3298421491,6917530014031375339,Legendary,Leg Armor,1900,7,19,6,15,6,11,False,False,Titan,64,False,Titan Leg Armor
173,Greaves of Agony,3846650177,6917529874293774311,Legendary,Leg Armor,1900,8,21,2,9,6,14,False,True,Titan,60,False,Titan Leg Armor
174,Deep Explorer Mark,420895300,6917529837730519359,Legendary,Class Item,1900,0,0,0,0,0,0,True,True,Titan,0,False,Titan Class Item
175,Descending Echo Mark,3500810712,6917529559439886508,Legendary,Class Item,1900,0,0,0,0,0,0,True,True,Titan,0,False,Titan Class Item
